### import environment

In [10]:
import gradio as gr
import tempfile
import os
import google.generativeai as genai
import json
from PIL import Image
import time  
import random
import io
import streamlit as st  
import sqlite3  
from transformers import pipeline

In [11]:
emotion_classifier = pipeline("text-classification", model="j-hartmann/emotion-english-distilroberta-base", return_all_scores=True)
model = genai.GenerativeModel("gemini-1.5-flash", safety_settings="BLOCK_NONE",
        system_instruction="You are a mushroom expert, only reply with answers related to mushrooms")
#Starting a chat so we can give the bot memory using the history
chat = model.start_chat(history=[])

### define a promt to convert user input into SQL queue

In [12]:
def prompt_to_generate_sql(user_input):
    prompt = f"""
    You are an expert in converting English questions to SQL queries!
    The SQL database has one table. The name of the table is 'songs'.
    
    The 'songs' table has the following columns: 
    'id', 'title', 'artist', 'album', 'release_year', 'genre', 'duration_ms',
    'lyrics', 'popularity', 'danceability', 'energy', 'valence', 'acousticness',
    'instrumentalness', 'liveness', 'loudness', 'speechiness', 'tempo'
    
    Your job is to write the SQL query based on a given prompt.

    Example user question: Show me the titles of all songs by 'Taylor Swift'.
    SQL query: SELECT title FROM songs WHERE artist = 'Taylor Swift';
    
    Example user question: {user_input}
    """
    return prompt

### connect to sqlite database

In [13]:
import sqlite3

# Function to execute SQL query and get the result
def execute_sql_query(sql_query):
    conn = sqlite3.connect('songs.db')  # Connect to your database
    cursor = conn.cursor()
    
    cursor.execute(sql_query)  # Execute the generated SQL query
    result = cursor.fetchall()  # Fetch all results from the query
    
    conn.close()  # Close the database connection
    return result

### Step 3: SQL Result to User-Friendly Response (LLM generates user-friendly response)

In [14]:
# Define a prompt to convert SQL results into a user-friendly response
def prompt_for_friendly_response(sql_result):
    prompt = f"""
    You are an expert in converting SQL results into user-friendly responses.
    Here's the SQL result: {sql_result}
    
    Your task is to generate a descriptive response based on the data. Make sure the response is meaningful and easy to understand.

    For example, if the result shows the total number of songs or the average duration of songs, mention it clearly. If it shows the titles of songs, list them with proper formatting.
    """
    return prompt

### define a prompt to convert user input into an sql using llm


In [15]:
def generate_sql_query(user_input, temperature =0.7, top_k=50, top_p = 0.9):
    model = genai.GenerativeModel("gemini-1.5-flash")
    prompt = f"""
    You are an expert at converting user input into SQL queries.
    Given the user's input:"{user_input}", write an appropriate SQL query to fetch the relevant data.
    """
    response = model.send_message(prompt, generation_config = genai.types.GenerationConfig(
        temperature = temperature,
        top_k =top_k,
        top_p =top_p
    ))
    sql_query = response.text.strip()
    return sql_query

### get mood


In [16]:
def get_mood(text_input):
    # Get mood predictions
    moods = emotion_classifier(text_input)[0]
    # Sort moods by score (confidence)
    moods_sorted = sorted(moods, key=lambda x: x['score'], reverse=True)
    # Get the top mood with the highest confidence
    mood = moods_sorted[0]['label']
    return mood
get_mood("i want cry")

'sadness'

### become a lyrics expert

In [17]:
# Initialize the chatbot as a Lyrics Expert
def become_lyrics_expert():
    # Act as a Lyrics Expert
    converted_history.append({"role": "user", "parts": "You are a lyrics expert. Answer only lyrics-related questions."})
    converted_history.append({"role": "model", "parts": "Understood! I’m here to provide insights and suggestions based on song lyrics."})

    # User level
    converted_history.append({"role": "user", "parts": "You're talking to someone who loves analyzing lyrics."})
    converted_history.append({"role": "model", "parts": "Great! I’ll provide detailed analyses and song recommendations based on lyrics."})

    # Keywords in lyrics
    converted_history.append({"role": "user", "parts": "The user is looking for songs containing specific keywords."})
    converted_history.append({"role": "model", "parts": "I’ll suggest songs that contain the provided keywords in their lyrics."})

    # Positive or negative preference
    converted_history.append({"role": "user", "parts": "The user has a preference for songs with positive or negative lyrics."})
    converted_history.append({"role": "model", "parts": "I’ll provide song recommendations based on the user's preference for either positive or negative lyrics."})

    # Sensitive topic (suicide)
    converted_history.append({"role": "user", "parts": "The user may express suicidal thoughts."})
    converted_history.append({"role": "model", "parts": "I’ll respond with positive messages and suggest uplifting songs to provide support and encouragement."})

    # No small talk
    converted_history.append({"role": "user", "parts": "Do not engage in small talk."})
    converted_history.append({"role": "model", "parts": "Understood! I’ll focus solely on lyrics-related inquiries without casual conversation."})

    # Solely Lyrics
    converted_history.append({"role": "user", "parts": "Only discuss lyrics analysis. No way to talk about anything else."})
    converted_history.append({"role": "model", "parts": "I’ll ensure all responses are strictly related to analyzing lyrics, song suggestions, and recommendations."})

# Add to the history
def convert_last_history(converted_history, last_history):
    user_message, model_response = last_history
    if user_message and model_response:
        converted_history.append({"role": "user", "parts": user_message})
        converted_history.append({"role": "model", "parts": model_response})


### function1 Song selection based on keywords

In [18]:
def get_songs_by_keywords(keywords):
    # Mocked song database
    song_database = [
        {"title": "Happy", "artist": "Pharrell Williams", "lyrics": "Because I'm happy..."},
        {"title": "Sad", "artist": "XXX", "lyrics": "I'm feeling so sad..."},
        {"title": "Joy", "artist": "XYZ", "lyrics": "Bring me joy..."}
    ]
    # Filter songs containing the keyword in the lyrics
    matched_songs = [song for song in song_database if any(keyword.lower() in song["lyrics"].lower() for keyword in keywords)]
    return matched_songs

###  Function 2: Song selection based on mood preference


In [19]:
def get_songs_by_mood(preference):
    # Mocked song database with positive/negative sentiments
    song_database = [
        {"title": "Happy", "artist": "Pharrell Williams", "mood": "positive", "lyrics": "Because I'm happy..."},
        {"title": "Depressed", "artist": "Sad Band", "mood": "negative", "lyrics": "I feel so low..."},
        {"title": "Joy", "artist": "XYZ", "mood": "positive", "lyrics": "Bring me joy..."}
    ]
    # Filter songs by mood preference
    matched_songs = [song for song in song_database if song["mood"] == preference]
    return matched_songs


### Function 3: Sensitive topic detection and positive response


In [20]:
def respond_to_sensitive_topics(user_message):
    if "suicide" in user_message.lower():
        positive_response = {
            "response": "I'm really sorry you're feeling this way, but I'm here to support you. Here's a positive song that might lift your spirits.",
            "suggested_songs": get_songs_by_mood("positive")
        }
        return positive_response
    return None

### example usage

In [21]:
def handle_lyrics_keywords(keywords, db_conn):
    if not keywords:  # Check if keywords list is empty
        return "Please provide a keyword to search."

    try:
        # Prepare the SQL query to search for songs based on lyrics
        keyword_clause = " OR ".join([f"lyrics LIKE ?" for _ in keywords])
        sql_query = f"SELECT title, artist FROM songs WHERE {keyword_clause}"
        
        # Prepare the parameters for the SQL query
        keyword_params = [f"%{keyword}%" for keyword in keywords]  # Use wildcards for partial matching

        # Execute the query
        cursor = db_conn.cursor()
        cursor.execute(sql_query, keyword_params)
        matching_songs = cursor.fetchall()

        # Generate the response based on the matched songs
        if matching_songs:
            song_list = ', '.join([f"{song[0]} by {song[1]}" for song in matching_songs])
            return f"I found songs that match the keyword '{keywords[0]}': {song_list}."
        else:
            return f"Sorry, I couldn't find any songs containing the keyword '{keywords[0]}'."
    
    except sqlite3.Error as e:
        return f"Database error: {e}"

### def of the model and chat

### simulate streamed response 

In [22]:
def simulate_typing_response(answer):
    full_sentence = ""
    for word in answer.split():
        full_sentence += word + " "
        yield full_sentence.strip()  
        time.sleep(0.3)  

### rewind function

In [23]:
def rewind_chat(history):
    if len(history) > 0:
        last_safe_state = history[-1]  # Get the last safe state
        print("Rewinding to the last safe state:", last_safe_state)
        return history[:-1]  
    else:
        return history  

In [24]:
def response(inputs, history, temperature=0.7, top_k=50, top_p=0.9):
    try:
        # If prompt is empty
        if inputs['text'] == '' and inputs['files'] == []:
            return "Please provide text input or upload an image."

        # If prompt is simply text, handle it by generating a SQL query
        if inputs['files'] == [] and inputs['text'] != '':
            user_input = inputs["text"].lower()

            if "hello" in user_input:
                return "Hello! I am a melody and song database expert. Ask me anything about songs or artists!"
            elif "bye" in user_input:
                return "Goodbye! Have a great day!"

            try:
                # Detect mood from user input using the get_mood function
                mood = get_mood(user_input)
                if mood:
                    response_text = handle_lyrics_mood(mood)
                    return response_text

                # If no specific mood is detected, convert the user's input into an SQL query using the model
                sql_query = generate_sql_query(user_input, temperature, top_k, top_p)
                
                # Execute the generated SQL query
                sql_result = execute_sql_query(sql_query)
                
                # Generate a friendly response from the SQL result
                if sql_result:
                    user_friendly_response = prompt_for_friendly_response(sql_result)
                else:
                    user_friendly_response = "Sorry, no matching data found in the database."
                
                return user_friendly_response

            except Exception as e:
                return f"Error processing your request: {e}"

        # Handle image-based inputs (use similar logic to your existing image handling)
        elif inputs['files'] != []:
            # Placeholder for image handling logic
            im = Image.open(inputs['files'][0]['path'])
            p = model.generate_content([
                "Provide a JSON answer with the following information: song title, artist, "
                "confidence of song match (number between 0 and 1), genre, mood (positive/negative), "
                "is this song recommended for your mood (Boolean)?", im
            ], generation_config=genai.types.GenerationConfig(
                temperature=temperature,
                top_k=top_k,
                top_p=top_p
            )).text

            # Summarize the image information if no additional text is provided
            if inputs['text'] == '':
                return model.generate_content(
                    ["Summarize briefly the information in", p],
                    generation_config=genai.types.GenerationConfig(
                        temperature=temperature,
                        top_k=top_k,
                        top_p=top_p
                    )
                ).text
            else:
                try:
                    return model.generate_content(
                        [inputs['text'], im],
                        generation_config=genai.types.GenerationConfig(
                            temperature=temperature,
                            top_k=top_k,
                            top_p=top_p
                        )
                    ).text
                except Exception as e:
                    return f"Your prompt triggered a safety filter. Error: {e}"

    except Exception as e:
        return f"An error occurred: {e}"

# Helper functions

# Function to handle keywords in lyrics
def handle_lyrics_keywords(keywords):
    if not keywords:
        return "Please provide a keyword to search."

    try:
        # SQL query to search for songs by lyrics
        keyword_clause = " OR ".join([f"lyrics LIKE '%{keyword}%'" for keyword in keywords])
        sql_query = f"SELECT title, artist FROM songs WHERE {keyword_clause}"

        # Use the execute_sql_query function to get the result
        matching_songs = execute_sql_query(sql_query)

        if matching_songs:
            song_list = ', '.join([f"{song[0]} by {song[1]}" for song in matching_songs])
            return f"I found songs that match the keyword '{keywords[0]}': {song_list}."
        else:
            return f"No songs found containing the keyword '{keywords[0]}'."
    
    except sqlite3.Error as e:
        return f"Database error: {e}"

# Function to handle mood-based song recommendations
def handle_lyrics_mood(mood):
    try:
        # SQL query to search for songs based on mood
        sql_query = f"SELECT title, artist FROM songs WHERE mood = '{mood}'"
        
        # Use the execute_sql_query function to get the result
        matching_songs = execute_sql_query(sql_query)

        if matching_songs:
            song_list = ', '.join([f"{song[0]} by {song[1]}" for song in matching_songs])
            return f"Here are some {mood} songs: {song_list}."
        else:
            return f"No {mood} songs found."

    except sqlite3.Error as e:
        return f"Database error: {e}"
# Simulate typing response
def simulate_typing_response(text):
    if not text:
        text = "Ask me about music!"
    for i in range(0, len(text), 10):
        yield text[:i + 10]

### interface


In [26]:
import gradio.themes as theme  # Import Gradio themes

# Create the Gradio app with a theme and examples
with gr.Blocks(theme=theme, fill_height=True) as demo:
    # Define a ChatInterface with multimodal support
    chatbot = gr.ChatInterface(
        multimodal=True,    # Supports text and images
        fn=response,        # The function that processes the user input
        title="🎵 Your best Melody Expert 🎵",  # Title of the chatbot
    )
    
if __name__ == "__main__":
    # Launch the Gradio app with public sharing enabled
    demo.launch(share=True)

/opt/conda/lib/python3.11/site-packages/gradio/blocks.py:985: UserWarning: Theme should be a class loaded from gradio.themes
  warnings.warn("Theme should be a class loaded from gradio.themes")


Running on local URL:  http://0.0.0.0:7862


/opt/conda/lib/python3.11/site-packages/gradio/analytics.py:106: UserWarning: IMPORTANT: You are using gradio version 4.43.0, however version 5.0.1 is available, please upgrade. 
--------
  warnings.warn(


Running on public URL: https://2b9f4946947d30df4f.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
